# Assignment: CKY Parsing

## Natural Language Processing

## Name: Richard Lee

### Overview

In this assignment, we will be building a simple parser that is trained from the ATIS (Air Traffic Information System) portion of the Penn Treebank. The training data consists of short queries and commands spoken by users of a fake robot travel agent.

The files provided and their purpose are listed below:

    - `train.trees.pre.unk` contains the trees of the training data: binary branching trees with all words that only appear once replaced with `<unk>`  
    - `dev.strings` contains the Strings of the development data
    - `dev.trees` contains the trees of the development data
    - `test.strings` contains the Strings of the test data
    - `test.trees` contains the trees of the test data
    - `preprocess.py` contains the preprocessor
    - `postprocess.py` contains the postprocessor
    - `evalb.py` contains a script to compute the labeled precision, recall, and F1 
    - `trees.py` is used by other scripts

### Setup
    
Import the `trees` package and read the first line of `train.trees.pre.unk` into a variable named `line`. Then, run the following cell to display the tree. The line `trees.Tree.from_str(line)` will be a useful line of code in this assignment.

In [1]:
import trees

with open('./train.trees.pre.unk', 'r') as file:
    line = file.readline().strip()


In [2]:
sample_tree = trees.Tree.from_str(line)
print(sample_tree.pretty_print())

                                   TOP
         ┌──────────────────────────┴──────────────────────────┐
        S_VP                                                  PUNC
  ┌──────┴───────┐                                             │
  VB             NP                                            .
  │    ┌─────────┴─────────┐
List   NP                 NP*
    ┌──┴──┐       ┌────────┴─────────┐
    DT   NNS      PP                NP*
    │     │    ┌──┴───┐       ┌──────┴───────┐
   the flights IN   NP_NNP    PP            SBAR
               │      │     ┌─┴──┐       ┌───┴────┐
             from Baltimore TO NP_NNP WHNP_WDT   S_VP
                            │    │       │     ┌──┴──┐
                           to Seattle  that   VBP    PP
                                               │  ┌──┴───┐
                                             stop IN   NP_NNP
                                                  │      │
                                                 in Minneapolis


### Training

Write code to learn a probabilistic context-free grammar from the tree data and store it in the following format:

    NP -> DT NN, 0.5
    NP -> DT NNS, 0.5
    DT -> the, 1.0
    NN -> boy, 0.5
    NN -> girl, 0.5
    NNS -> boys, 0.5
    NNS -> girls, 0.5
    
To do this, create a `get_rules` function that reads in the trees from the `train.trees.pre.unk` file and stores the rules found in these trees in a dictionary called `rules`, with the rule as the key, and its count as the value. Run the following cell to run your function on the training data and report the number of unique rules found as well as the top 5 most frequent rules, along with their counts. (It is highly recommended that you study the trees.py code.)

In [3]:
def get_rules(filename):
    
    with open(filename) as f:
        lines = f.readlines()

    rules = {}

    for line in lines:
        tree = trees.Tree.from_str(line)

        # Use the Tree class in trees.py
        for node in tree.bottomup():
            if len(node.children) == 1:
                RHS = node.children[0].label
            elif len(node.children) < 1:
                continue
            elif len(node.children) == 2:
                RHS = node.children[0].label + " " + node.children[1].label

            rule = "" + node.label + " -> " + RHS
            if rule in rules.keys():
                rules[rule] = rules[rule] + 1
            else:
                rules[rule] = 1

    return rules

In [4]:
rules = get_rules("train.trees.pre.unk")

ordered_rules_list = list(sorted(rules.items(), key=lambda x:x[1], reverse=True))
print("There are " + str(len(rules.keys())) + " unique rules in the file.")
print("The 5 most frequent rules and their counts are: ")
for i in range(5):
    print(ordered_rules_list[i])

There are 754 unique rules in the file.
The 5 most frequent rules and their counts are: 
('PUNC -> .', 346)
('TO -> to', 241)
('PP -> IN NP_NNP', 239)
('IN -> from', 218)
('PP -> IN NP', 197)


Now, write a function that takes a dictionary of rules and their counts and creates a new dictionary called `rule_probs` that stores the conditional probability of each rule. For example, if there are two rules for `NP` which are both seen an equal number of times in the training data, both of their probabilities would be 50%. Run the following cell to run your function on the rules dictionary and report the top five highest probability rules with left-hand side NNP as well as their probabilities.

In [5]:

def get_probs(rules):

    probs = {}
    lhs_counts = {}

    # Calculate the total counts for each LHS
    for rule in rules:
        lhs = rule.split(' -> ')[0]
        lhs_counts[lhs] = lhs_counts.get(lhs, 0) + rules[rule]

    # Calculate probabilities for each rule
    for rule in rules:
        lhs = rule.split(' -> ')[0]
        probs[rule] = rules[rule] / lhs_counts[lhs]

    return probs


In [6]:
rule_probs = get_probs(rules)

NNP_rule_probs = {}
for rule in rule_probs:
    rule_split = rule.split(' -> ')
    if rule_split[0] == 'NNP':
        NNP_rule_probs[rule] = rule_probs[rule]

ordered_NNP_rule_probs = list(sorted(NNP_rule_probs.items(), key=lambda x:x[1], reverse=True))
print('There were ' + str(len(NNP_rule_probs.keys())) + ' unique NNP rules found.')
print('The 5 most frequent NNP rules were: ')
for i in range(5):
    print(ordered_NNP_rule_probs[i])

There were 62 unique NNP rules found.
The 5 most frequent NNP rules were: 
('NNP -> San', 0.07352941176470588)
('NNP -> City', 0.07107843137254902)
('NNP -> New', 0.061274509803921566)
('NNP -> Las', 0.058823529411764705)
('NNP -> Vegas', 0.058823529411764705)


### Parsing

Now write a `CKY` function that takes a sentence and the grammar (the `rule_probs`) as input and runs the CKY algorithm to fill a chart with the highest probability partial parses. Return the filled in chart.

In [7]:

def CKY(sent, rule_probs):
    words = sent.split()
    n = len(words)

    chart = [[{} for _ in range(n + 1)] for _ in range(n)]

    for j in range(1, n + 1):
        word = words[j - 1]
        added = False
        for rule, prob in rule_probs.items():
            lhs, rhs = rule.split(" -> ")
            rhs_symbols = rhs.split()
            if len(rhs_symbols) == 1 and rhs_symbols[0] == word:
                chart[j - 1][j][lhs] = (prob, None, word)
                added = True

        if not added:
            for rule, prob in rule_probs.items():
                lhs, rhs = rule.split(" -> ")
                if rhs == "<unk>":
                    chart[j - 1][j][lhs] = (prob, None, word) 

    for span in range(2, n + 1):
        for begin in range(n - span + 1):
            end = begin + span
            for split in range(begin + 1, end):
                for rule, prob in rule_probs.items():
                    lhs, rhs = rule.split(" -> ")
                    rhs_symbols = rhs.split()
                    if len(rhs_symbols) == 2:
                        Y, Z = rhs_symbols
                        if Y in chart[begin][split] and Z in chart[split][end]:
                            prob_Y = chart[begin][split][Y][0]
                            prob_Z = chart[split][end][Z][0]
                            prob_total = prob_Y * prob_Z * prob
                            if lhs not in chart[begin][end] or chart[begin][end][lhs][0] < prob_total:
                                chart[begin][end][lhs] = (prob_total, split, Y, Z) 

    return chart



Write a recursive `parse` function that takes in the CKY chart, the s token which is used for recursion, a row index, and a column index, and outputs the highest-probability parse. If you can’t find any parse, output a blank line. Don’t forget to replace unknown words with `<unk>`. If you encounter underflow, consider using log probabilities. Run the following cells to test your CKY parser on the dev set.

In [8]:

def parse(chart, s='TOP', row=0, col=-1):
    if col == -1:
        col = len(chart)

    if row >= len(chart) or col > len(chart[row]) or s not in chart[row][col]:
        return ""

    entry = chart[row][col][s]
    if entry[1] is None: 
        return f"({s} {entry[2]})"  

    split, Y, Z = entry[1:]
    left_parse = parse(chart, Y, row, split)
    right_parse = parse(chart, Z, split, col)

    return f"({s} {left_parse} {right_parse})"




In [9]:
# Read in dev.strings
with open("dev.strings") as f:
    dev_lines = f.read()
dev = dev_lines.split('\n')
    
# Run parse on dev.strings
dev_parses = []
for i in range(len(dev)):
    sent = dev[i]
    my_chart = CKY(sent, rule_probs)
    my_parse = parse(my_chart, 'TOP', 0, -1)
    
    dev_parses.append(my_parse)
    
# Save parses to dev.parses
j = open("dev.parses", "w")
for tempparse in dev_parses:
    j.write(tempparse + '\n')
j.close()


Run the next cell to generate `dev.pcfg_parses`. Do not chance the code.


In [10]:
!python postprocess.py dev.parses > dev.pcfg_parses

Finally, run the last cell to report the first 5 lines of `dev_strings` with the probabilities.

In [11]:
# Report first 5 lines of dev_strings with probabilities
for i in range(5):
    sent = dev[i]
    my_chart = CKY(sent, rule_probs)
    if 'TOP' in my_chart[0][-1].keys():
        prob = my_chart[0][-1]['TOP'][0]
    else:
        prob = 0
    my_parse = parse(my_chart, 'TOP', 0, -1)
    print('Line ' + str(i) + ": " + str(dev[i]))
    print('Parse: ' + str(my_parse))
    print('Probability: ' + str(prob))
    print('-----------------------------')




Line 0: The flight should be eleven a.m tomorrow .
Parse: 
Probability: 0
-----------------------------
Line 1: I would like it to have a stop in New York and I would like a flight that serves breakfast .
Parse: (TOP (S (S (NP_PRP I) (VP (MD would) (VP (VB like) (S (NP_PRP it) (VP (TO to) (VP (VBP have) (NP (NP (DT a) (NN stop)) (PP (IN in) (NP (NNP New) (NNP York)))))))))) (S* (CC and) (S (NP_PRP I) (VP (MD would) (VP (VB like) (NP (NP (DT a) (NN flight)) (SBAR (WHNP_WDT that) (S_VP (VBZ serves) (ADVP_RB breakfast))))))))) (PUNC .))
Probability: 9.290910822145571e-38
-----------------------------
Line 2: Which of these serve dinner ?
Parse: (TOP (SBARQ (WHNP (WHNP_WDT Which) (PP (IN of) (NP_DT these))) (SQ_VP (VBP serve) (NP_NN dinner))) (PUNC ?))
Probability: 1.6783381809002082e-09
-----------------------------
Line 3: Which ones stop in Nashville ?
Parse: (TOP (SBARQ (WHNP (WDT Which) (NNS ones)) (SQ_VP (VBP stop) (PP (IN in) (NP_NNP Nashville)))) (PUNC ?))
Probability: 5.9779816120

.

In [12]:
list(rule_probs.items())[:5]

[('VB -> List', 0.018072289156626505),
 ('DT -> the', 0.6262626262626263),
 ('NNS -> flights', 0.7464114832535885),
 ('NP -> DT NNS', 0.08550724637681159),
 ('IN -> from', 0.45228215767634855)]

In [13]:
rule_probs['WHNP_WDT -> Which']

0.5714285714285714

In [14]:
rule_probs['IN -> of']

0.08091286307053942

In [15]:
rule_probs['NP_DT -> these']

0.5333333333333333

In [16]:
rule_probs['VBP -> serve']

0.043859649122807015

In [17]:
rule_probs['NP_NN -> dinner']

0.27450980392156865

In [18]:
rule_probs['PUNC -> ?']

0.2622601279317697

Finally, run the following cells to test your parser and `postprocess.py` again on `test.strings` and save the output to `test.pcfg_parses`and to evaluate your parser output against the correct trees in `test.trees`. The F1 score in the output of the script should be at least 90%.

In [19]:
# Read in test.strings
with open("test.strings") as f:
    test_lines = f.read()
test = test_lines.split('\n')

# Run parse on test.strings
test_parses = []
for i in range(len(test)):
    sent = test[i]
    my_chart = CKY(sent, rule_probs)
    my_parse = parse(my_chart, 'TOP', 0, -1)
    
    test_parses.append(my_parse)

# Save parses to test.parses
f = open("test.parses", "w")
for tempparse in test_parses:
    f.write(tempparse + '\n')
f.close()

Run the next cells to generate the file `eval.model`.
You can open it to check that your F1 score is greater than 85%.

Do not change the code


In [20]:
!python postprocess.py test.parses > test.pcfg_parses

In [21]:
!python evalb.py test.pcfg_parses test.trees > eval.model

.